In [1]:
'''Trains a LSTM on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


In [8]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [12]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [13]:
%%time
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 62s - loss: 0.4543 - acc: 0.7840 - val_loss: 0.3830 - val_acc: 0.8314
Epoch 2/15
25000/25000 [==============================] - 62s - loss: 0.2922 - acc: 0.8807 - val_loss: 0.3854 - val_acc: 0.8324
Epoch 3/15
25000/25000 [==============================] - 62s - loss: 0.2078 - acc: 0.9209 - val_loss: 0.4099 - val_acc: 0.8300
Epoch 4/15
25000/25000 [==============================] - 62s - loss: 0.1388 - acc: 0.9489 - val_loss: 0.4936 - val_acc: 0.8336
Epoch 5/15
25000/25000 [==============================] - 62s - loss: 0.0964 - acc: 0.9660 - val_loss: 0.5927 - val_acc: 0.8178
Epoch 6/15
25000/25000 [==============================] - 62s - loss: 0.0749 - acc: 0.9741 - val_loss: 0.6644 - val_acc: 0.8226
Epoch 7/15
25000/25000 [==============================] - 62s - loss: 0.0644 - acc: 0.9780 - val_loss: 0.6556 - val_acc: 0.8193
Epoch 8/15
25000/25000 [======

Use LSTM.

Process data by getting overlapping segments of `maxlen` length subsequence of words as independent training samples.

Perform over sampling to classes with small absolute training data.

When making the predictions, split the text in N overlapping segments as well and average the predictions for each segment to get the likely class distribution.



In [14]:
model.predict_proba(x_train[:10])

10/10 [==============================] - 0s


array([[  9.99938130e-01],
       [  3.69773549e-03],
       [  4.69944971e-05],
       [  9.99998927e-01],
       [  2.76938827e-05],
       [  1.89894181e-05],
       [  9.99962568e-01],
       [  1.75047462e-05],
       [  9.99999642e-01],
       [  7.29953026e-05]], dtype=float32)

In [15]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])